# Rede Neural

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Visão Computacional

In [3]:
import cv2 
import numpy as np
import time

import PC as pc

In [31]:
tempoInicialGeral = 0
tempoFinalGeral = 0
tempoDeterminado = 5
tempoExecutando = 0

baixo = [2, 0, 0]
alto  = [30, 255, 255]

PASTA_RESULTADO = "FOLHAS_SEM_RESULTADOS/"
PASTA_RESULTADOS_ARQUIVOS = "DADOS_RESULTADOS/"
EXTENSAO = ".txt"
INFORMACAO = "TESTE"

info = []


idx = 1
qtdImagens = 251


arquivo = open(PASTA_RESULTADOS_ARQUIVOS + PASTA_RESULTADO[:-1] + "_" + INFORMACAO + EXTENSAO, "w")

media = []
hist = []

print("Executando...\n\n")

tempoInicialGeral = time.time()

while(idx <= qtdImagens):
        
    tempoExecuntandoInicial = time.time()
    
    img = cv2.imread(IMAGENS_VALIDACAO_PASTA + str(idx) + IMAGENS_VALIDACAO_ARQUIVO)
        
    image_np = np.array(img)
    
    category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')
        
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor) 

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    imagem_pedunculo = image_np.copy()
    
    detection_threshold = 0.7

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                instance_masks=detections.get('detection_masks'),
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.8,
                skip_scores= False,
                skip_labels= True,
                #skip_boxes= False,
                agnostic_mode=True)
    
    tempoExecuntandoFinal = time.time()
    tempoExecutando = tempoExecutando + (tempoExecuntandoFinal - tempoExecuntandoInicial)
    
    #print(tempoExecutando)
    
    if(tempoExecutando > tempoDeterminado):
        
        print("{} - Fruta nao identificada".format(idx))
        info.append(str(idx) + ",0,0")
        idx += 1
    
    scores = list(filter(lambda x: x> detection_threshold, 
    detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
        
    media.append(scores)

    width = image_np_with_detections.shape[1]
    height = image_np_with_detections.shape[0]
    
    try:
        
        for idxb, box in enumerate(boxes):

            roi = box * [height, width, height, width]

            xt = roi[1] #- 120     #linha esquerda
            xb = roi[3]           #linha direita
            yt = roi[0] #- 130     #linha superior
            yb = roi[2]           #linha inferior

            #Chama a função que prevê a área do pedúnculo
            areaPedunculo = pc.preveAreaPedunculo(xt, yt, xb, yb, 0.3, 0.5, -10)

            x1, y1, x2, y2 = areaPedunculo[0], areaPedunculo[1], areaPedunculo[2], areaPedunculo[3]

            #Marca um quadrado na região do pedúnculo        
            cv2.rectangle(image_np_with_detections, (x1,y1), (x2,y2), (50,0,255), 3)

            TopLeftX = x1
            TopLeftY = y2

            #Faz o corte na área prevista do pedúnculo
            corteAreaPedunculo = imagem_pedunculo[y2:y1, x1:x2]  

            #cv2.imwrite(" " + str(idx) + "_ponto_3_cadidatos.jpg", corteAreaPedunculo) 

            #Chama a função para encontrar o ponto de corte        
            coordenadaFinal = pc.coordenadaPontoFinal(corteAreaPedunculo,
                                                          baixo,
                                                          alto,
                                                          TopLeftX, 
                                                          TopLeftY, 
                                                          "Kmeans", 
                                                          7,
                                                         2)

            pontoX, pontoY, areaSegmentada, pontosCandidatos, valorHue = coordenadaFinal[0], coordenadaFinal[1], coordenadaFinal[2], coordenadaFinal[3], coordenadaFinal[4]

            print("{} - Fruta identificada - HUE: {}".format(idx, valorHue))
            info.append(str(idx) + ",1," + str(valorHue))
            
            cv2.circle(image_np_with_detections, (pontoX, pontoY), 10, (0,0,255), -1)
            
            cv2.line(image_np_with_detections, (pontoX, pontoY), (int((xt+xb)/2), int(yt)), (0,255,0), 3)

            cv2.circle(image_np_with_detections, (int((xt+xb)/2), int(yt)), 10, (255,0,90), -1)

            #cv2.imwrite(PASTA_RESULTADO + str(idxb) + "_area_1_pedunculo.jpg", corteAreaPedunculo)        
            #cv2.imwrite(PASTA_RESULTADO + str(idxb) + "_area_2_segmentada.jpg", areaSegmentada)        
            cv2.imwrite(PASTA_RESULTADO + str(idx) + "_ponto_cadidatos.jpg", pontosCandidatos)        
            cv2.imwrite(PASTA_RESULTADO + str(idx) + "_ponto_final.jpg", image_np_with_detections)

            idx = idx + 1
            tempoExecutando = 0
    except:
        
        print("{} - Fruta muito proxima".format(idx))
        info.append(str(idx) + ",-0,-0")
        
        cv2.imwrite(PASTA_RESULTADO + str(idx) + "_ponto_final.jpg", image_np_with_detections)
        
        idx = idx + 1
        tempoExecutando = 0
        
tempoFinalGeral = time.time()

print("\n\nFinalizando...")
print("Tempo de execucao: {}".format((tempoFinalGeral - tempoInicialGeral)))


Executando...


1 - Fruta identificada - HUE: 29
2 - Fruta identificada - HUE: 29
3 - Fruta identificada - HUE: 24
4 - Fruta identificada - HUE: 25
5 - Fruta identificada - HUE: 29
6 - Fruta identificada - HUE: 24
7 - Fruta identificada - HUE: 25
8 - Fruta identificada - HUE: 25
9 - Fruta identificada - HUE: 26
10 - Fruta identificada - HUE: 16
11 - Fruta identificada - HUE: 17
12 - Fruta identificada - HUE: 11
13 - Fruta identificada - HUE: 11
Media central
14 - Fruta identificada - HUE: 0
Media central
15 - Fruta identificada - HUE: 0
16 - Fruta identificada - HUE: 29
17 - Fruta identificada - HUE: 29
18 - Fruta identificada - HUE: 19
19 - Fruta identificada - HUE: 19
20 - Fruta identificada - HUE: 20
21 - Fruta identificada - HUE: 13
22 - Fruta identificada - HUE: 13
23 - Fruta identificada - HUE: 14
24 - Fruta identificada - HUE: 4
Media central
25 - Fruta identificada - HUE: 0
26 - Fruta identificada - HUE: 29
27 - Fruta identificada - HUE: 29
Area do pedunculo sem informacao
28 -

Media central
231 - Fruta identificada - HUE: 3
232 - Fruta identificada - HUE: 4
233 - Fruta identificada - HUE: 29
234 - Fruta identificada - HUE: 9
235 - Fruta identificada - HUE: 3
236 - Fruta identificada - HUE: 3
237 - Fruta identificada - HUE: 29
238 - Fruta identificada - HUE: 4
239 - Fruta identificada - HUE: 6
240 - Fruta identificada - HUE: 7
241 - Fruta identificada - HUE: 12
242 - Fruta identificada - HUE: 11
243 - Fruta identificada - HUE: 17
244 - Fruta identificada - HUE: 17
245 - Fruta identificada - HUE: 16
246 - Fruta identificada - HUE: 9
247 - Fruta identificada - HUE: 9
248 - Fruta identificada - HUE: 18
249 - Fruta identificada - HUE: 18
250 - Fruta identificada - HUE: 18
251 - Fruta identificada - HUE: 17


Finalizando...
Tempo de execucao: 457.75012469291687


In [44]:
soma = 0

informa = []

for i in info:
    
    informa.append(i)
    
print(informa)



['1,1,29', '2,1,29', '3,1,24', '4,1,25', '5,1,29', '6,1,24', '7,1,25', '8,1,25', '9,1,26', '10,1,16', '11,1,17', '12,1,11', '13,1,11', '14,1,0', '15,1,0', '16,1,29', '17,1,29', '18,1,19', '19,1,19', '20,1,20', '21,1,13', '22,1,13', '23,1,14', '24,1,4', '25,1,0', '26,1,29', '27,1,29', '28,0,0', '29,0,0', '30,1,0', '31,1,0', '32,1,2', '33,1,21', '34,1,23', '35,1,23', '36,1,9', '37,1,13', '38,1,13', '39,1,13', '40,1,8', '41,1,9', '42,1,9', '43,1,9', '44,1,29', '45,1,29', '46,1,29', '47,1,29', '48,1,29', '49,1,25', '50,1,28', '51,1,27', '52,1,27', '53,1,25', '54,1,23', '55,1,23', '56,1,27', '57,1,23', '58,1,21', '59,1,21', '60,1,18', '61,1,21', '62,1,21', '63,1,22', '64,1,23', '65,1,23', '66,1,23', '67,1,27', '68,1,27', '69,1,27', '70,1,14', '71,1,14', '72,1,16', '73,1,13', '74,1,20', '75,1,20', '76,1,21', '77,1,13', '78,1,13', '79,1,18', '80,1,11', '81,1,29', '82,1,11', '83,1,14', '84,1,14', '85,1,11', '86,1,11', '87,1,25', '88,1,9', '89,1,9', '90,1,9', '91,1,29', '92,1,29', '93,0,0', '94